In [13]:
import ee
import geemap
import os
ee.Authenticate()


Successfully saved authorization token.


In [14]:
os.environ['HTTP_PROXY']='http://127.0.0.1:7890'
os.environ['HTTPS_PROXY']='http://127.0.0.1:7890'
ee.Initialize()

In [15]:
#导入数据集
dataset_TER = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')

In [19]:
Map = geemap.Map()

# 加载中国的行政边界数据（已经上传到 GEE）
china_boundary = ee.FeatureCollection("projects/chancy479/assets/province")

GX = china_boundary.filterMetadata('ID','equals','Guangxi')

Map.center_object(GX,6)
Map.addLayer(GX, {'color': 'FF0000'}, 'Guangxi Boundary')
Map

Map(center=[23.81821645035185, 108.78433096092434], controls=(WidgetControl(options=['position', 'transparent_…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [35]:
#导入并筛选
dataset_LANDSAT=ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
filtered_LANDSAT = dataset_LANDSAT.filterBounds(GX).filterDate('2022-07-01', '2022-8-15')
print('图片数量：', filtered_LANDSAT.size().getInfo())

图片数量： 48


In [33]:
#时间信息
# 获取图像的时间信息
image=filtered_LANDSAT.first()
time_start = image.get('system:time_start')

# 打印时间信息（以毫秒为单位）
print("Time Start (Milliseconds):", time_start.getInfo())

# 将时间戳转换为可读日期时间格式（以UTC时间为准）
import datetime
date = datetime.datetime.fromtimestamp(time_start.getInfo() / 1000.0)
print("Date:", date)

Time Start (Milliseconds): 1658631494206
Date: 2022-07-24 10:58:14.206000


In [62]:
# 计算NDVI函数
def calculate_ndvi(image):
    NDVI=image.normalizedDifference(['B5', 'B4']).rename('NDVI')    
    return image.addBands(NDVI)

L8_NDVI=filtered_LANDSAT.map(calculate_ndvi)
print(L8_NDVI.getInfo())

{'type': 'ImageCollection', 'bands': [], 'version': 1694075000659597, 'id': 'LANDSAT/LC08/C02/T1_TOA', 'features': [{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7581, 7731], 'crs': 'EPSG:32649', 'crs_transform': [30, 0, 524985, 0, -30, 2831715]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7581, 7731], 'crs': 'EPSG:32649', 'crs_transform': [30, 0, 524985, 0, -30, 2831715]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7581, 7731], 'crs': 'EPSG:32649', 'crs_transform': [30, 0, 524985, 0, -30, 2831715]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7581, 7731], 'crs': 'EPSG:32649', 'crs_transform': [30, 0, 524985, 0, -30, 2831715]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7581, 7731], 'crs': 'EPSG:32649', 'crs_transform': [30, 0, 524985, 0, -30, 2831715

In [64]:
bandList = ee.List(ee.Dictionary(ee.Algorithms.Describe(L8_NDVI.first())).get('bands'))
print(bandList.getInfo())

[{'crs': 'EPSG:32649', 'crs_transform': [30, 0, 524985, 0, -30, 2831715], 'data_type': {'precision': 'float', 'type': 'PixelType'}, 'dimensions': [7581, 7731], 'id': 'B1'}, {'crs': 'EPSG:32649', 'crs_transform': [30, 0, 524985, 0, -30, 2831715], 'data_type': {'precision': 'float', 'type': 'PixelType'}, 'dimensions': [7581, 7731], 'id': 'B2'}, {'crs': 'EPSG:32649', 'crs_transform': [30, 0, 524985, 0, -30, 2831715], 'data_type': {'precision': 'float', 'type': 'PixelType'}, 'dimensions': [7581, 7731], 'id': 'B3'}, {'crs': 'EPSG:32649', 'crs_transform': [30, 0, 524985, 0, -30, 2831715], 'data_type': {'precision': 'float', 'type': 'PixelType'}, 'dimensions': [7581, 7731], 'id': 'B4'}, {'crs': 'EPSG:32649', 'crs_transform': [30, 0, 524985, 0, -30, 2831715], 'data_type': {'precision': 'float', 'type': 'PixelType'}, 'dimensions': [7581, 7731], 'id': 'B5'}, {'crs': 'EPSG:32649', 'crs_transform': [30, 0, 524985, 0, -30, 2831715], 'data_type': {'precision': 'float', 'type': 'PixelType'}, 'dimensi